In [2]:
if 'NOTEBOOK_INITIATED_FLAG' not in globals():
    NOTEBOOK_INITIATED_FLAG = True
    %cd ..
# %pwd

from _1_preprocessing._3_feature_engineering import score_df

c:\projects\aws


In [3]:
import json
import sys
import time
import pickle
from collections import Counter

import pandas as pd
import requests

url = "https://axieinfinity.com/graphql-server-v2/graphql"

Query

In [4]:
query = """
query GetAxieBriefList($auctionType: AuctionType, $criteria: AxieSearchCriteria, $from: Int, $sort: SortBy, $size: Int, $owner: String) {
  axies(auctionType: $auctionType, criteria: $criteria, from: $from, sort: $sort, size: $size, owner: $owner) {
    results {
      ...AxieBrief
    }
  }
}

fragment AxieBrief on Axie {
  id
  class
  breedCount
  image
  auction {
    currentPrice
    currentPriceUSD
  }
  battleInfo {
    banned
  }
  parts {
    name
    class
    type
    specialGenes
  }
}
"""
def variables(fromm):
    return {
        "from": fromm,
        "size": 100,
        "sort": "PriceAsc",
        "auctionType": "Sale",
        "owner": None,
        "criteria": {
            "region": None,
            "parts": None,
            "bodyShapes": None,
            "classes": None,
            "stages": None,
            "numMystic": None,
            "pureness": None,
            "title": None,
            "breedable": None,
            "breedCount": None,
            "hp": [],
            "skill": [],
            "speed": [],
            "morale": [],
        },
    }

Filter (ml model) to apply<br>
One hot encoder for the class column<br>
Card and combo scores

In [5]:
with open('./_2_model_training/_RF.pkl', 'rb') as f: # RF, KNN, , tree, polynomial, SVM
    model = pickle.load(f)
with open("./_1_preprocessing/one_hot_encoder.pickle", "rb") as f:
    oh_enc = pickle.load(f)
with open("./_1_preprocessing/scores_lookup.txt") as f:
    for i in f.readlines():
        scores_lookup = i
scores_lookup = eval(scores_lookup)

Do the request and measure performance

In [8]:
def compare_two_prices(row, id_):
    if (row['Price'] +100 < row['Prediction']) and (row['Price']>50):
        global n_found
        n_found += 1
        # print(id_)
        # print('Axie card score: ',row['sum_card_score'])

def compare_prices(price_comparaison):
    price_comparaison.apply(lambda df: compare_two_prices(df, df.name), axis=1)

start = time.time()
request = requests.post(
            url,
            json={"query": query, "variables": variables(0)},
            verify=False,
        )
end = time.time()
print("Time elapsed doing the request: ", end - start, "s\n")
# Check the data retrieved
if request.status_code == 200:
    jsn_data = request.json()
else:
    raise Exception(f"Unexpected status code returned: {request.status_code}")
jsn_data = request.json()["data"]["axies"]["results"]
#
jsn = []
n_found = 0
not_parsed_axies = 0
if jsn_data:
    info_batch=pd.DataFrame()
    price_batch=pd.DataFrame()
    image_batch=pd.DataFrame()    
    for ax in jsn_data:
        # print(json.dumps(ax, indent=4, sort_keys=True))
        if ax["battleInfo"]["banned"] == False:
            try:
                id_ = ax["id"]
                class_ = ax["class"]
                image = ax["image"]
                price = float(ax["auction"]["currentPriceUSD"])
                breedCount = ax["breedCount"]
                eyes = ax["parts"][0]["name"]
                ears = ax["parts"][1]["name"]
                back = ax["parts"][2]["name"]
                mouth = ax["parts"][3]["name"]
                horn = ax["parts"][4]["name"]
                tail = ax["parts"][5]["name"]
                eyes_type = ax["parts"][0]["class"]
                ears_type = ax["parts"][1]["class"]
                back_type = ax["parts"][2]["class"]
                mouth_type = ax["parts"][3]["class"]
                horn_type = ax["parts"][4]["class"]
                tail_type = ax["parts"][5]["class"]
                axie_info = pd.DataFrame({
                    "BreedCount": breedCount,
                    "Class": class_,
                    "Eyes": eyes,
                    "Ears": ears,
                    "Back": back,
                    "Mouth": mouth,
                    "Horn": horn,
                    "Tail": tail,
                    "EyesType": eyes_type,
                    "EarsType": ears_type,
                    "BackType": back_type,
                    "MouthType": mouth_type,
                    "HornType": horn_type,
                    "TailType": tail_type,
                }, index=[id_])
                start = time.time()
                info_batch = info_batch.append(axie_info)
                price_batch = price_batch.append(pd.DataFrame({"Price": price},index=[id_]))
                image_batch = image_batch.append(pd.DataFrame({"Image": image}, index=[id_]))
                end = time.time()
                print("Time elapsed appending one axies: ", end - start, "s")
            except:
                not_parsed_axies += 1

    start = time.time()
    axie_values = score_df(info_batch, scores_lookup, class_encoder=oh_enc)
    end = time.time()
    print("Time elapsed on feature engineering a hundred axies: ", end - start, "s")

    start = time.time()
    price_predictions = axie_values.apply(lambda row: model.predict(row.values.reshape(1,-1))[0], axis=1)
    end = time.time()
    print("Time elapsed on infering the price of a hundred axies: ", end - start, "s")

    # start = time.time()
    price_comparaisons = pd.concat([axie_values,price_batch, price_predictions, image_batch], axis=1).rename({0: "Prediction"},axis=1)
    compare_prices(price_comparaisons)
    # end = time.time()
    # print("Time elapsed comparing prices: ", end - start, "s")

jsn = json.dumps(jsn, indent=2)
print("Estimated size: " + str(sys.getsizeof(jsn) / 1024) + "KB")
print(f"Axies failed to  parse: ",not_parsed_axies,"/",len(jsn_data) )
print(f"Amount of bargains: {n_found}")
# print(jsn)


c:\projects\aws\venv\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'axieinfinity.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Time elapsed doing the request:  2.6912641525268555 s

Time elapsed appending one axies:  0.0029997825622558594 s
Time elapsed appending one axies:  0.006065845489501953 s
Time elapsed appending one axies:  0.003996849060058594 s
Time elapsed appending one axies:  0.00400090217590332 s
Time elapsed appending one axies:  0.00304412841796875 s
Time elapsed appending one axies:  0.001960277557373047 s
Time elapsed appending one axies:  0.001974821090698242 s
Time elapsed appending one axies:  0.0010063648223876953 s
Time elapsed appending one axies:  0.002003192901611328 s
Time elapsed appending one axies:  0.0019986629486083984 s
Time elapsed appending one axies:  0.001996755599975586 s
Time elapsed appending one axies:  0.0009899139404296875 s
Time elapsed appending one axies:  0.001954793930053711 s
Time elapsed appending one axies:  0.0020036697387695312 s
Time elapsed appending one axies:  0.0009946823120117188 s
Time elapsed appending one axies:  0.0019419193267822266 s
Time elapsed